##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 학습 알고리즘 작성하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/composing_learning_algorithms"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.36.0/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 시작하기 전에

시작하기 전에 다음을 실행하여 환경이 올바르게 설정되었는지 확인하세요. 인사말이 표시되지 않으면 [설치](../install.md) 가이드에서 지침을 참조하세요. 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from typing import Callable

import tensorflow as tf
import tensorflow_federated as tff

**참고**: 이 Colab은 <code>tensorflow_federated</code> pip 패키지의 <a>최신 릴리즈 버전</a>에서 동작하는 것으로 확인되었지만 Tensorflow Federated 프로젝트는 아직 시험판 개발 중이며 `main`에서 동작하지 않을 수 있습니다.

# 학습 알고리즘 작성

[고유한 페더레이션 학습 알고리즘 구축 튜토리얼](https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb)에서는 TFF의 페더레이션 코어를 사용하여 페더레이션 평균화(FedAvg) 알고리즘 버전을 직접 구현했습니다.

이 튜토리얼에서는 TFF API의 페더레이션 학습 구성 요소를 사용하여 모든 것을 처음부터 다시 구현할 필요 없이 모듈 방식으로 페더레이션 학습 알고리즘을 구축합니다.

이 튜토리얼의 목적을 위해 로컬 학습을 통해 그래디언트 클리핑을 사용하는 FedAvg의 변형을 구현합니다.

## 학습 알고리즘 빌딩 블록

고차원적 수준에서 많은 학습 알고리즘은 **빌딩 블록**이라고 하는 4개의 개별 구성 요소로 분리될 수 있으며, 다음과 같습니다.

1. 배포자(즉, 서버-클라이언트 통신)
2. 클라이언트 작업(예: 로컬 클라이언트 계산)
3. 집계자(예: 클라이언트-서버 통신)
4. 종결자(즉, 집계된 클라이언트 출력을 사용하는 서버 계산)

[고유한 페더레이션 학습 알고리즘 구축 튜토리얼](https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb)이 이러한 모든 구성 요소를 처음부터 구현했지만 그럴 필요가 없는 경우가 종종 있습니다. 대신 유사한 알고리즘의 빌딩 블록을 재사용할 수 있습니다.

이 경우 그래디언트 클리핑으로 FedAvg를 구현하려면 **클라이언트 작업** 빌딩 블록만 수정하면 됩니다. 나머지 블록은 "바닐라" FedAvg에 사용된 것과 동일할 수 있습니다.

# 클라이언트 작업 구현하기

먼저, 그래디언트 클리핑으로 로컬 모델 훈련을 수행하는 TF 로직을 작성해 보겠습니다. 단순하게 하기 위해 그래디언트는 최대 1의 표준을 가집니다.

## TF 로직

In [ ]:
@tf.function
def client_update(model: tff.learning.Model,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

위의 코드에는 몇 가지 주목할 부분들이 있습니다. 첫째, 클라이언트 업데이트의 *가중치*를 구성할 것이므로 본 예제의 수를 추적합니다(클라이언트 전체의 평균을 계산할 때).

둘째, [`tff.learning.templates.ClientResult`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientResult)를 사용하여 출력을 패키징합니다. 이 반환 유형은 `tff.learning`에서 클라이언트 작업의 빌딩 블록을 표준화하는 데 사용됩니다.

## ClientWorkProcess 만들기

위의 TF 로직은 클리핑을 사용하여 로컬 훈련을 수행하지만 필요한 빌딩 블록을 생성하려면 여전히 TFF 코드로 래핑해야 합니다.

구체적으로, 4개의 빌딩 블록이 [`tff.templates.MeasuredProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/MeasuredProcess)로 표시됩니다. 이것은 4개의 블록 모두가 계산을 인스턴스화하고 실행하는 데 사용되는 `initialize` 및 `next` 함수를 모두 가지고 있음을 의미합니다.

이를 통해 각 빌딩 블록은 작업을 수행하는 데 필요한 자체 **상태**(서버에 저장됨)를 추적할 수 있습니다. 이 튜토리얼에서는 사용되지 않지만, 얼마나 많은 반복이 이루어졌는지 추적하거나 옵티마이저 상태를 추적하는 등에 이를 사용할 수 있습니다.

클라이언트 작업 TF 로직은 일반적으로 [`tff.learning.templates.ClientWorkProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientWorkProcess)로 래핑되어야 하며, 이는 클라이언트의 로컬 훈련에 들어오고 나가는 예상 유형을 코드화합니다. 이는 아래와 같이 모델과 옵티마이저에 의해 매개변수화될 수 있습니다.

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.Model],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.framework.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# 학습 알고리즘 작성

위의 클라이언트 작업을 본격적인 알고리즘에 넣어 보겠습니다. 먼저 데이터와 모델을 설정하겠습니다.

## 입력 데이터 준비하기

TFF에 포함된 EMNIST 데이터세트를 로드하고 전처리합니다. 자세한 내용은 [이미지 분류](federated_learning_for_image_classification.ipynb) 튜토리얼을 참조하세요.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

데이터세트를 모델에 제공하기 위해 데이터가 병합되고 `(flattened_image_vector, label)` 형식의 튜플로 변환됩니다.

소수의 클라이언트를 선택하고 위의 전처리를 해당 데이터세트에 적용해 보겠습니다.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## 모델 준비하기

여기서는 [이미지 분류](federated_learning_for_image_classification.ipynb) 튜토리얼에서와 동일한 모델이 사용됩니다. 이 모델(`tf.keras`를 통해 구현됨)에는 하나의 숨겨진 레이어, 그 다음 소프트맥스 레이어가 있습니다. TFF에서 이 모델을 사용하기 위해 Keras 모델은 [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model)로 래핑됩니다. 이를 통해 TFF 내에서 모델의 [정방향 전달](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass)을 수행하고 [모델 출력을 추출](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics)할 수 있습니다. 자세한 내용은 [이미지 분류](federated_learning_for_image_classification.ipynb) 튜토리얼을 참조하세요.

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## 옵티마이저 준비하기

[`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg)에서와 마찬가지로 여기에는 클라이언트 옵티마이저와 서버 옵티마이저라는 두 가지 옵티마이저가 있습니다. 단순화를 위해 옵티마이저는 학습률이 다른 SGD가 됩니다.

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## 빌딩 블록 정의하기

이제 클라이언트 작업 빌딩 블록, 데이터, 모델 및 옵티마이저가 설정되었으므로 배포자, 집계자 및 종결자를 위한 빌딩 블록을 만드는 일만 남았습니다. 이를 수행하기 위해 TFF에서 사용할 수 있고 FedAvg에서 사용하는 일부 기본값을 차용할 수 있습니다.

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## 빌딩 블록 구성하기

마지막으로, 빌딩 블록을 결합하기 위해 TFF의 내장 **작성기**를 사용할 수 있습니다. 이것은 위의 4가지 빌딩 블록을 사용하고 그 유형을 함께 연결하는 비교적 간단한 작성기입니다.

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# 알고리즘 실행하기

이제 알고리즘이 완료되었으므로 실행해 보겠습니다. 먼저, 알고리즘을 **초기화**합니다. 이 알고리즘의 **상태**에는 *전역 모델 가중치*에 대한 구성 요소와 함께 각 빌딩 블록에 대한 구성 요소가 있습니다.

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

예상대로 클라이언트 작업은 비어 있는 상태입니다(위의 클라이언트 작업 코드를 상기할 것!). 그러나 다른 빌딩 블록은 비어 있지 않은 상태를 가질 수 있습니다. 예를 들어 종결자는 얼마나 많은 반복이 이루어졌는지 추적합니다. `next`는 아직 실행되지 않았으므로 상태는 `0`입니다.

In [ ]:
state.finalizer

[0]

이제 훈련 라운드를 실행합니다.

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

이 출력(`tff.learning.templates.LearningProcessOutput`)에는 `.state` 및 `.metrics` 출력이 모두 있습니다. 둘 다 살펴보겠습니다.

In [ ]:
learning_process_output.state.finalizer

[1]

분명히 종결자 상태는 `.next` 한 라운드가 실행됨에 따라 1씩 증가했습니다.

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

메트릭은 비어 있지만 더 복잡하고 실용적인 알고리즘의 경우 일반적으로 유용한 정보로 가득 차게 됩니다.

# 결론

위의 빌딩 블록/작성기 프레임워크를 사용하면 모든 작업을 처음부터 다시 수행할 필요 없이 완전히 새로운 학습 알고리즘을 만들 수 있습니다. 그러나 이것은 시작일 뿐입니다. 이 프레임워크를 사용하면 알고리즘을 FedAvg의 간단한 변형으로 훨씬 쉽게 표현할 수 있습니다. 더 많은 알고리즘을 보려면 [`tff.learning.algorithms`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms)를 참조하세요. 여기에는 [FedProx](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) 및 [클라이언트 학습률 스케줄링이 있는 FedAvg](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule)과 같은 알고리즘이 포함되어 있습니다. 이러한 API는 [페더레이션 k-평균 클러스터링](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans)과 같은 완전히 새로운 알고리즘의 구현을 지원할 수도 있습니다.